In [1]:
import numpy as np
import pandas as pd
import math
import statistics
import matplotlib.pyplot as plt
from transformers import pipeline
from sklearn.model_selection import train_test_split, cross_val_score,KFold, cross_val_predict, GridSearchCV,RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.datasets import make_classification
from sklearn.neural_network import MLPClassifier


In [2]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

## Download All Policies - Train Set

In [3]:
##Privacy Policies download:
websites = ['Google','Aws','AliExpress','Meta','TikTok','YouTube','Waze','Wix','Bookings','whatsapp'
            ,'apple','wolt','Visa','Mastercard','AirBNB','uber','Spotify','samsung','Wordpress','instagram'
            ,'MacDonalds','FDA','Oracle','Zara','cocacola','Xiaomi','Nasdaq','Walmart'
           ,'AirCanada','Lufthansa','shopify','Netflix','adobe','Starbucks','Shoppers','Decathlon','waltdisney'
            ,'AmericanEagle','lululemon','SAP','JetBrains','MySQLCode','Cadens','EpicGames'
            ,'unitedHealthGroup','Slack','SalesForce','JPMorgan','JohnsonAndJohnson']

sequence_to_classify = []

for i in websites:
    try:
        with open((i+".txt"), "r") as f:
            i = f.readlines()
            sequence_to_classify.append(i)
    except:
        with open((i+".txt"), "r", encoding='cp1252') as f:
            i = f.readlines()
            sequence_to_classify.append(i)

### Train Set Vectors:

In [4]:
user_edit_data_train = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,0,1,1,0,1]

### Edit & Delete data

In [5]:
##Edit delete data Matrix:

Phrases = ['Privacy Policy'
           ,'YOU CAN EDIT AND DELETE YOUR DATA ANY TIME'
           ,'If your personally identifiable information changes, or if you no longer desire our service, you may update it or request deletion by contacting us'
           ,'you have only partial ability to edit and delete your personal data'
           ,'You can choose to delete, change or correct your personal information or access your personal information'
           ,'You have the right to request that we delete certain personal information we have collected from you']

##Matrix creation:
matrix_edit_delete_ML = pd.DataFrame(columns = Phrases)
matrix_edit_delete_ML['Privacy Policy'] = websites
matrix_edit_delete_ML.set_index('Privacy Policy', inplace=True)

##Filling the matrix:
line = 0
for j in sequence_to_classify:
    for i in matrix_edit_delete_ML:
        matrix_edit_delete_ML.loc[websites[line],i] = classifier(str(j), str(i))["scores"][0]
    line += 1
    
matrix_edit_delete_ML


,YOU CAN EDIT AND DELETE YOUR DATA ANY TIME,"If your personally identifiable information changes, or if you no longer desire our service, you may update it or request deletion by contacting us",you have only partial ability to edit and delete your personal data,"You can choose to delete, change or correct your personal information or access your personal information",You have the right to request that we delete certain personal information we have collected from you
Privacy Policy,,,,,
Google,0.919693,0.469185,0.661442,0.586523,0.904135
Aws,0.52252,0.497995,0.586619,0.633738,0.316978
AliExpress,0.598408,0.446928,0.2373,0.830736,0.409912
Meta,0.858651,0.449768,0.744424,0.674782,0.957846
TikTok,0.540718,0.516206,0.66241,0.68332,0.706546
YouTube,0.700698,0.507361,0.668441,0.618786,0.926949
Waze,0.730179,0.433804,0.660415,0.569268,0.899602
Wix,0.52864,0.498468,0.375705,0.640079,0.667173
Bookings,0.88843,0.388308,0.758334,0.728894,0.935611


### Test The Model - ML

In [6]:
# X_train, X_test, y_train, y_test = train_test_split(matrix_policy_change, policy_change_train
#                                                     , test_size=0.5, random_state=42)

# Manual division so we will have all options (1/0) at both groups
X_train = matrix_edit_delete_ML.iloc[:35,:] 
X_test = matrix_edit_delete_ML.iloc[34:,:]
y_train = user_edit_data_train[:35]
y_test = user_edit_data_train[34:]

##Decission Tree Classifaier
regressor_tree = DecisionTreeClassifier()
regressor_tree = regressor_tree.fit(X_train, y_train)
pred_tree = regressor_tree.predict(X_test)
print('Decission Tree Classifaier:', pred_tree)
print(classification_report(y_test, pred_tree))

##Random Forest Classifaier
regressor_forest = RandomForestClassifier()
regressor_forest = regressor_forest.fit(X_train, y_train)
pred_forest = regressor_forest.predict(X_test).round()
print('Random Forest Classifaier:', pred_forest)
print(classification_report(y_test, pred_forest))

##Logistic Reg Classifier
clf = LogisticRegression()
clf.fit(X_train, y_train)
pred_clf = clf.predict(X_test)
print('Logistic Reg Classifier:', pred_clf)
print(classification_report(y_test, pred_clf))

##SVM
SVM = SVC(kernel='linear')
SVM = SVM.fit(X_train, y_train)
pred_SVM = SVM.predict(X_test)
print('SVM:', pred_SVM)
print(classification_report(y_test, pred_SVM))

##KNN
KNN = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p=2)
KNN = KNN.fit(X_train, y_train)
pred_KNN = KNN.predict(X_test)
print('KNN:', pred_KNN)
print(classification_report(y_test, pred_KNN))

##neural_network
neural_network = MLPClassifier(random_state=1, max_iter=300)
neural_network = neural_network.fit(X_train, y_train)
pred_neural_network = neural_network.predict(X_test)
print('neural_network:', pred_neural_network)
print(classification_report(y_test, pred_neural_network))

Decission Tree Classifaier: [0 1 1 1 1 1 1 1 0 0 1 1 1 1 1]
              precision    recall  f1-score   support

           0       0.33      0.33      0.33         3
           1       0.83      0.83      0.83        12

    accuracy                           0.73        15
   macro avg       0.58      0.58      0.58        15
weighted avg       0.73      0.73      0.73        15

Random Forest Classifaier: [0 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
              precision    recall  f1-score   support

           0       1.00      0.33      0.50         3
           1       0.86      1.00      0.92        12

    accuracy                           0.87        15
   macro avg       0.93      0.67      0.71        15
weighted avg       0.89      0.87      0.84        15

Logistic Reg Classifier: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.80      1.00      0.89        12

    ac

/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalb

In [7]:
##Third Party Transfer Matrix:

##Matrix creation:
matrix_edit_delete_Comparison = pd.DataFrame(columns = ['Phrase',
                                               'Google','Aws','AliExpress','Meta','TikTok','YouTube','Waze','Wix','Bookings','whatsapp'
            ,'apple','wolt','Visa','Mastercard','AirBNB','uber','Spotify','samsung','Wordpress','instagram'
            ,'MacDonalds','FDA','Oracle','Zara','cocacola','Xiaomi','Nasdaq','Walmart'
           ,'AirCanada','Lufthansa','shopify','Netflix','adobe','Starbucks','Shoppers','Decathlon','waltdisney'
            ,'AmericanEagle','lululemon','SAP','JetBrains','MySQLCode','Cadens','EpicGames'
            ,'unitedHealthGroup','Slack','SalesForce','JPMorgan','JohnsonAndJohnson'])
matrix_edit_delete_Comparison['Phrase'] = ['Third Party Transfer']

##Filling the matrix:
websites_index = 0
line = 0
pred = 0
edit_delete_train_pred_Comparison = []
c = ['YOU CAN EDIT AND DELETE YOUR DATA ANY TIME','you have a partial ability to edit and delete your data','YOU CAN not EDIT AND DELETE YOUR DATA ANY TIME']

for j in sequence_to_classify:
    a = classifier(str(j),c)
    if a["labels"][0]==c[0]:# and a["scores"][0]>0.6:
        pred = 1
    elif a["labels"][0]==c[1]:
        pred = 2
    else:
        pred = 3  
    matrix_edit_delete_Comparison[websites[websites_index]].loc[line] = pred
    edit_delete_train_pred_Comparison.append(pred)
    websites_index += 1
    pred = 0

edit_delete_train_pred_Comparison

[2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2]

In [8]:
index = 0
TP = 0
FP = 0
FN = 0
TN = 0
for i in user_edit_data_train:
    if edit_delete_train_pred_Comparison[index]==1 and user_edit_data_train[index]==1:
        TP +=1
    elif edit_delete_train_pred_Comparison[index]==0 and user_edit_data_train[index]==0:
        TN +=1
    elif edit_delete_train_pred_Comparison[index]==0 and user_edit_data_train[index]==1:
        FN +=1
    else:
        FP +=1
    index +=1

recall = TP/(TP+FN)
specificity = TN/(TN+FP)
Precision = TP/(TP+FP)
Accuracy = (TP+TN)/(TP+FP+FN+TN) 
F1 = (TP)/(TP+(0.5*(FP+FN)))

print("recall:" , recall)
print("specificity:" , specificity)
print("Precision:" , Precision)
print("Accuracy:" , Accuracy)
print("F1 Score:" , F1)

recall: 1.0
specificity: 0.0
Precision: 0.061224489795918366
Accuracy: 0.061224489795918366
F1 Score: 0.11538461538461539
